In [ ]:
import os
import numpy as np
import pandas as pd
import os
import torch
import warnings
from .dataloader.dataloader import DatasetCustom
from .models import DLinear, NLinear, PatchTST
from .models.stat_models import Arima
from torch.utils.data import DataLoader
import json
from dataloader.api import *
from pandas.tseries.offsets import BusinessDay
import plotly.graph_objects as go
warnings.filterwarnings('ignore')

In [ ]:
dir()

In [ ]:
import json
import pandas as pd
data = json.load(open('../datasets/data.json'))

df = pd.DataFrame(data["data"])
cols = ['rowDate',
    'volumeRaw',
    'last_closeRaw',
    'last_openRaw',
    'last_maxRaw',
    'last_minRaw',
    'change_precentRaw']
data = df[cols].copy()
data.rowDate = pd.to_datetime(data.rowDate, format="%d/%m/%Y")
data.loc[:, "volumeRaw":] = data.loc[:, "volumeRaw":].astype("Float32")
print(data.info())
data.head()
data = data.sort_values("rowDate", ascending=True).reset_index(drop=True)
# data.to_parquet("./datasets/VNINDEX_2013-01-02_2023-09-11.parquet", index=False)
data.head()

In [ ]:
symbol = "VNINDEX"
# data_path = f"{symbol}_2000-07-28_09-11-2023.parquet"
data_path = f"{symbol}_2013-01-02_2023-09-11.parquet"
# data = get_historical_price_fireant(symbol, "01-01-2000", "09-11-2023")


In [ ]:
data = pd.read_parquet(f"./datasets/{data_path}")
# data.rowDate = pd.to_datetime(data.rowDate, format="%Y/%m/%d").dt.strftime("%Y/%m/%d")
# data.rowDate = pd.to_datetime(data.rowDate, format="%Y/%m/%d")

In [ ]:
data.info()

In [ ]:
import pmdarima as pm
pm.autocorr_plot(data.last_closeRaw);
pm.plot_pacf(data.last_closeRaw)

In [ ]:
from types import SimpleNamespace

configs = SimpleNamespace(
    task_name="short_term_forecast",
    batch_size=512,
    is_training=1,
    seq_len=96, # 96  36 48 60 72 144 288
    pred_len=96, # 96 192 336
    label_len=0, # 48, 18
    seasonal_patterns=None, # not used in this dataset
    moving_avg=25, # 25 default
    embed="timeF",
    activation="gelu",
    output_attention=False,
    learning_rate=0.001,
    epoch=30,
    freq="D",
    enc_in=1, # number of features, 1 in case forecasting
    num_class=1, # in case classification
    individual=True,
    scale=True,
    time_enc=1,
    features="S", # ["S", "M", "MS"]
    # transformer model
    d_model=16,
    e_layers=3,
    d_ff=128,
    c_out=1,
    factor=3,
    n_heads=4,
    dropout=0.1, # 0.3 for patchTST
    # fc_dropout=0.3, # dont use
    head_dropout=0,
    patch_len = 16,
    stride = 8
)

time_configs = SimpleNamespace(
    task_name="short_term_forecast",
    batch_size=256,
    is_training=1,
    seq_len=24, # 96  36 48 60 72 144 288
    pred_len=24, # 96 192 336
    label_len=0, # 48, 18
    epoch=100,
    freq="D",
    enc_in=1, # number of features, 1 in case forecasting
    num_class=1, # in case classification
    individual=True,
    scale=True,
    time_enc=1,
    features="S", # ["S", "M", "MS"]
    sample=1,
)

In [ ]:
train_test_ratio = (0.95, 0.05)
train_dataset = Dataset_Custom(
    "datasets", 
    flag="train",
    features=configs.features, 
    data_path=data_path, 
    target="last_closeRaw", 
    timeenc=configs.time_enc, 
    freq=configs.freq,
    scale=configs.scale,
    size=(configs.seq_len, configs.label_len, configs.pred_len),
    train_test_ratio = train_test_ratio,
    # train_only=True
)
val_dataset = Dataset_Custom(
    "datasets", 
    flag="val",
    features=configs.features, 
    data_path=data_path, 
    target="last_closeRaw", 
    timeenc=configs.time_enc, 
    freq=configs.freq,
    scale=configs.scale,
    size=(configs.seq_len, configs.label_len, configs.pred_len),
    train_test_ratio = train_test_ratio
)
test_dataset = Dataset_Custom(
    "datasets", 
    flag="test",
    features=configs.features, 
    data_path=data_path, 
    target="last_closeRaw", 
    timeenc=configs.time_enc, 
    freq=configs.freq,
    scale=configs.scale,
    size=(configs.seq_len, configs.label_len, configs.pred_len),
    train_test_ratio = train_test_ratio
)

train_dataloader= DataLoader(train_dataset, batch_size=configs.batch_size, drop_last=True, shuffle=True)
val_dataloader= DataLoader(val_dataset, batch_size=configs.batch_size, drop_last=False, shuffle=False)
test_dataloader= DataLoader(test_dataset, batch_size=configs.batch_size, drop_last=False, shuffle=False)

In [ ]:
from models import *

def get_model(name, configs):
    models = dict(
        dlinear = DLinear(configs),
        nlinear = NLinear(configs),
        patchtst = PatchTST(configs),
        arima = Arima(time_configs)
    )
    return models[name]
model = get_model("dlinear", configs)
optimizer = torch.optim.Adam(model.parameters(), lr=configs.learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.5)
best_state = {}
best_val_loss = float("inf")
best_epoch = -1
f_dim = 0 if configs.features=="S" else -1

In [ ]:
from copy import deepcopy
for epoch in range(configs.epoch):
    model.train()
    train_losses = []
    for batch in train_dataloader:
        seq_x, seq_y, seq_x_mark, seq_y_mark = batch
        optimizer.zero_grad(set_to_none=True)
        seq_x = seq_x.to(torch.float32)
        seq_y = seq_y.to(torch.float32)
        pred = model(seq_x, seq_x_mark, seq_y, seq_y_mark)[:, -configs.pred_len:, f_dim:]
        seq_y = seq_y[:, -configs.pred_len:, f_dim:]
        loss = torch.nn.functional.mse_loss(pred, seq_y)
        train_losses.append(loss.detach().item())
        loss.backward()
        optimizer.step()
    scheduler.step()
    print(f"{epoch=} {np.mean(train_losses)=:.5f}")
    if len(test_dataset) > 0:
        with torch.no_grad():
            model.eval()
            val_loss = []
            test_forecast = []
            for batch in test_dataloader:
                seq_x, seq_y, seq_x_mark, seq_y_mark = batch
                seq_x = seq_x.to(torch.float32)
                seq_y = seq_y.to(torch.float32)
                pred = model(seq_x, seq_x_mark, seq_y, seq_y_mark)[:, -configs.pred_len:, f_dim:]
                seq_y = seq_y[:, -configs.pred_len:, f_dim:]
                loss = torch.nn.functional.mse_loss(pred, seq_y).item()
                val_loss.append(loss)
                test_forecast.append(pred[0,:,:])
            val_loss = np.mean(val_loss)
            if val_loss < best_val_loss:
                best_epoch = epoch
                best_val_loss = val_loss
                best_state = deepcopy(model.state_dict())
            train_loss = np.mean(train_losses)
            print(f"{epoch=} {train_loss=:.5f} {val_loss=:.5f} {best_val_loss=:.5f} {best_epoch=}"  , sep="\n")


In [ ]:
test_forecast = []
if len(test_dataset) > 0:
    with torch.no_grad():
        model.eval()
        val_loss = []
        test_forecast = []
        for batch in test_dataloader:
            seq_x, seq_y, seq_x_mark, seq_y_mark = batch
            seq_x = seq_x.to(torch.float32)
            seq_y = seq_y.to(torch.float32)
            pred = model(seq_x, seq_x_mark, seq_y, seq_y_mark)[:, -configs.pred_len:, f_dim:]
            seq_y = seq_y[:, -configs.pred_len:, f_dim:]
            loss = torch.nn.functional.mse_loss(pred, seq_y).item()
            val_loss.append(loss)
            test_forecast.append(pred[0,:,:])
        val_loss = np.mean(val_loss)
        if val_loss < best_val_loss:
            best_epoch = epoch
            best_val_loss = val_loss
            best_state = deepcopy(model.state_dict())
        train_loss = np.mean(train_losses)
        print(f"{epoch=} {train_loss=:.5f} {val_loss=:.5f} {best_val_loss=:.5f} {best_epoch=}"  , sep="\n")

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error


plt.plot(train_dataset.inverse_transform(pred[0].numpy()))
plt.plot(train_dataset.inverse_transform(test_dataset.data_y[-configs.seq_len:][0:configs.pred_len]))
mean_squared_error(pred[0].numpy(), test_dataset.data_y[-configs.seq_len:][0:configs.pred_len])

In [ ]:
checkpoint = torch.load(f"./models/checkpoints/model_{symbol}_dlinear_96_96_full.pth")
model = DLinear(configs)
model.load_state_dict(checkpoint['model'])
# model.load_state_dict(best_state)

In [ ]:
scaler =  train_dataset.scaler
with torch.no_grad():
    model.eval()
    val_losses = []
    conformal_scores = []
    for batch in test_dataloader:
        seq_x, seq_y, seq_x_mark, seq_y_mark = batch
        seq_x = seq_x.to(torch.float32)
        seq_y = seq_y.to(torch.float32)
        pred = model(seq_x, seq_x_mark, seq_y, seq_y_mark)[:, -configs.pred_len:, f_dim:]
        seq_y = seq_y[:, -configs.pred_len:, f_dim:]
        loss = torch.nn.functional.mse_loss(pred, seq_y)
        val_losses.append(loss.item())
        pred = scaler.inverse_transform(pred.numpy().reshape(1,-1))[0]
        seq_y = scaler.inverse_transform(seq_y.numpy().reshape(1,-1))[0]
        conformal_scores.append(abs(pred - seq_y)/pred)
    print(f"test_mse_loss:{np.mean(val_losses):.5f}", sep="\n")
pi95 = np.quantile(np.concatenate(conformal_scores), q=0.95)
pi80 = np.quantile(np.concatenate(conformal_scores), q=0.8)
print(pi95, np.concatenate(conformal_scores).max(), np.concatenate(conformal_scores).min())

# Arima

In [ ]:
train_arima_dataset = Dataset_Custom(
    "datasets", 
    flag="train",
    features=configs.features, 
    data_path=data_path, 
    target="last_closeRaw", 
    timeenc=configs.time_enc, 
    freq=configs.freq,
    scale=configs.scale,
    size=(configs.seq_len, configs.label_len, configs.pred_len),
    train_test_ratio = train_test_ratio,
    train_only=True
)
# train_dataset.data_x = train_dataset.data_x[0, int(0.8)] 
# test_dataset = 

In [ ]:
from pmdarima.arima import ndiffs

kpss_diffs = ndiffs(train_dataset.data_x, alpha=0.05, test="kpss", max_d=6)
adf_diffs = ndiffs(train_dataset.data_x, alpha=0.05, test="adf", max_d=6)
n_diffs = max(adf_diffs, kpss_diffs)

print(f"Estimated differencing term: {n_diffs}")
# Estimated differencing term: 1
arima_model = pm.auto_arima(
    train_dataset.data_x,
    d=n_diffs,
    seasonal=True,
    stepwise=True,
    m=10,
    suppress_warnings=True,
    error_action="ignore",
    max_p=6,
    max_order=None,
    trace=True,
)
print(arima_model.order)

In [ ]:
from sklearn.metrics import mean_squared_error
from pmdarima.metrics import smape

def forecast_n_step(step=96):
    fc, conf_int = arima_model.predict(n_periods=step, return_conf_int=True)
    return (
        fc,
        np.asarray(conf_int).tolist())
arima_val, val_conf_int = forecast_n_step(len(val_dataset.data_x))
print(mean_squared_error(val_dataset.data_x, arima_val))

# forecasts = []
# confidence_intervals = []
# arima_losses = []
# for i, new_ob in enumerate(test_dataset.data_x[:-time_configs.pred_len]):
#     fc, conf = forecast_n_step(time_configs.pred_len)
#     confidence_intervals.append(conf)
#     # Updates the existing model with a small number of MLE steps
#     # because we dont have new_ob
#     loss = mean_squared_error(test_dataset.data_x[i:i+96], fc)
#     arima_losses.append(loss)
#     arima_model.update(new_ob)

# print(f"SMAPE: {smape(test_dataset.data_x, forecasts)}")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_dataset.inverse_transform(val_dataset.data_x).reshape(-1))
plt.plot(train_dataset.inverse_transform(arima_val.reshape(-1,1)).reshape(-1))

In [ ]:
for i, new_ob in enumerate(test_dataset.data_x[-time_configs.pred_len:]):
    arima_model.update(new_ob)
scaler.inverse_transform(test_dataset.data_x[-time_configs.pred_len:]).reshape(-1)
fc, conf = forecast_n_step(time_configs.pred_len)
arima_forcast = scaler.inverse_transform(fc.reshape(-1,1))
arima_forcast.reshape(-1)

# Future forecast

In [ ]:
from datetime import datetime, timedelta, timezone
from dataloader.api import get_historical_price_vnd

In [ ]:
tz = timezone(timedelta(hours=7))
start_time = int((data.rowDate.max() + BusinessDay(1)).timestamp())
end_time = int(datetime.now(tz=tz).timestamp())
forecast_gt_df = get_historical_price_vnd(symbol="VNINDEX", start_time=start_time, end_time=end_time)
forecast_gt_df.tail()

In [ ]:
model.eval()
with torch.no_grad():
    pred_seq_x, pred_seq_y, pred_seq_x_mark, pred_seq_y_mark = test_dataset[len(test_dataset.data_x) - configs.seq_len]
    pred_seq_x = torch.as_tensor(pred_seq_x)[None, :].to(torch.float32)
    pred_seq_y = torch.as_tensor(pred_seq_y)[None, :].to(torch.float32)
    pred_new = model(pred_seq_x, pred_seq_x_mark, pred_seq_y, pred_seq_y_mark)[:, -configs.pred_len:, 0:]
    print(pred_seq_x.shape, pred_new.shape)

In [ ]:
last_days = np.round(test_dataset.inverse_transform(pred_seq_x.numpy().reshape(1,-1))[0],2)
next_days = np.round(test_dataset.inverse_transform(pred_new.numpy().reshape(1,-1))[0],2)
next_days_pi95_low = np.round(next_days*(1-pi95),2)
next_days_pi95_high = np.round(next_days*(1+pi95),2) 
next_days_pi80_low = np.round(next_days*(1-pi80),2) 
next_days_pi80_high = np.round(next_days*(1+pi80),2) 

In [ ]:
pi95, pi80

In [ ]:
data.info()

In [ ]:
df_holiday = pd.read_csv("./datasets/vn_holiday_2025.csv")
df_holiday.head()

In [ ]:
last_date = data.rowDate.max() + BusinessDay(1)
pred_dates = pd.bdate_range(last_date, pd.Timestamp(last_date) + BusinessDay(configs.pred_len+ 50)) # add more 50 days due to holiday offline
pred_dates = list(filter(lambda x: x not in df_holiday["Date"].tolist(), pred_dates))
pred_dates = pred_dates[:configs.pred_len]
data_visual = data.query("rowDate >= '2022/01/01'")

colors = {
    "train": "rgb(48, 133, 195)",
    "forecast": "rgb(233, 184, 36)",
    "actual": "rgb(33, 156, 144)",
    "fill95": "rgba(255, 217, 61,0.1)",
    "fill80": "rgba(255, 217, 61,0.15)",
}

template="plotly_dark"
plot_data = []

plot_data.append(
    go.Scatter(
        x=data_visual.rowDate,
        y=data_visual.last_closeRaw,
        name="Training",
        legendrank=1,
        line=dict(color=colors["train"], width=2),
    )
)
plot_data.append(
    go.Scatter(
        x=pred_dates,
        y=next_days,
        name="Forecast",
        line=dict(color=colors["forecast"], width=2),
        legendrank=2
    )
)

plot_data.append(
    go.Scatter(
        x=forecast_gt_df.rowDate,
        y=forecast_gt_df.last_closeRaw,
        name="Actual",
        line=dict(color=colors["actual"], width=2),
        legendrank=3,
        mode="lines",
    )
)

plot_data.append(
    go.Scatter(
        x=pred_dates, 
        y=next_days_pi95_high,
        line=dict(color="rgba(0,0,0,0)"), 
        showlegend=False,
        hoverinfo="skip"
    )
)
plot_data.append(
    go.Scatter(
        x=pred_dates,
        y=next_days_pi95_low,
        name="Predict Interval 95th",
        fill="tonexty",
        fillcolor=colors["fill95"],
        line=dict(color="rgba(0,0,0,0)"),
        hoverinfo="skip",
    )
)
plot_data.append(
    go.Scatter(
        x=pred_dates, y=next_days_pi80_high, line=dict(color="rgba(0,0,0,0)"), showlegend=False, hoverinfo="skip"
    )
)
plot_data.append(
    go.Scatter(
        x=pred_dates,
        y=next_days_pi80_low,
        name="Predict Interval 80th",
        fill="tonexty",
        fillcolor=colors["fill80"],
        line=dict(color="rgba(0,0,0,0)"),
        hoverinfo="skip",
    )
)



layout = dict(
    title=dict(text=f"{symbol} LONGTERM FORECASTING"),
    hovermode="x unified",
    font_size=13,
    legend=dict(
        borderwidth=2,
    ),
    template=template,
    yaxis=dict(
        title=symbol,
        # linecolor="black" if "white" in template else "white",
        showgrid=True,
        gridwidth=1,
    ),
    xaxis=dict(
        # linecolor="black" if "white" in template else "white",
        title="Date",
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=6, label="6M", step="month", stepmode="backward"),
                    dict(count=1, label="1Y", step="year", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(step="all", label="ALL"),
                ]
            ),
            # font=dict(color="black"),
        ),
    ),
)
fig = go.Figure(data=plot_data, layout=layout)
fig.show()

In [ ]:
# torch.save(
#     {
#         "model": model.state_dict(),
#         "scaler": train_dataset.scaler
#     },
#     f"./models/checkpoints/model_{symbol}_dlinear_96_96_full.pth"
# )